# Lab 11 - Pruning Parcels

## Tasks

#### Task 1 - Proptype on 2004 parcel data

Build a query that will filter the 2004 parcel file

1. Read in the 2004 parce file
2. Create a column that contains True if the parcel is close to our 49 lakes (see lab 10) and flase otherwise. (Hint: create `lat_long` -> map to `code` -> check `isin` for the lakes we identified in lab 10).
3. Create a query that selects the correct columns, filters on the new column, then adds lake_names, lake_codes, and lake distance.  Finally filter by distance to the lake.

How much have we reduced the size of the parcel file?  Can you safely union all the files after these filters?

#### Task 2 - Read all 11 files and apply the process defined in the last task


#### Task 3 - Union all 11 files then write out the result.

In [2]:
import pandas as pd
from dfply import *
import datetime as dt
from glob import glob
import re
from toolz import first
from more_dfply import recode
from functoolz import pipeable

In [3]:
from project_cols_to_keep_and_drop import cols_to_keep

In [4]:
from project_raw_pd_parcel_types import common_parcel_types

In [5]:
from project_data_Miertschin import common_columns, ll_dist_dict, ll_code_dict, code_name_dict, ll_idnamedist_dict

In [6]:
files = glob('../MinneMUDAC_raw_files/20**_metro_tax_parcels.txt')[2:-1]

In [7]:
lake_stats = pd.read_csv("./data/lakes_stats.csv")

In [7]:
lake_stats.head()

,Unnamed: 0,DNR_ID_Site_Number,year,LAKE_NAME,mean_secchi,med_secchi,sd_secchi,mean_phos,med_phos,sd_phos
0,0,19002100-01,2004,Alimagnet Lake,0.445,0.500,0.204736,0.164500,0.1070,0.137039
1,1,19002100-01,2005,Alimagnet Lake,0.528,0.500,0.219484,0.123400,0.1275,0.038945
2,2,19002100-01,2006,Alimagnet Lake,0.525,0.500,0.185164,0.154375,0.1260,0.090448
3,3,19002100-01,2007,Alimagnet Lake,0.507,0.415,0.247792,0.124000,0.1125,0.064014
4,4,19002100-01,2008,Alimagnet Lake,0.605,0.600,0.252533,0.106167,0.1025,0.040580


In [8]:
lakes_2004 = pd.read_csv(files[0],sep='|',dtype = common_parcel_types)
lakes_2004.head()

,ACRES_DEED,ACRES_POLY,AGPRE_ENRD,AGPRE_EXPD,AG_PRESERV,BASEMENT,BLDG_NUM,BLOCK,CITY,CITY_USPS,...,XUSE1_DESC,XUSE2_DESC,XUSE3_DESC,XUSE4_DESC,YEAR_BUILT,Year,ZIP,ZIP4,centroid_lat,centroid_long
0,0.0,8.03,NaN,NaN,N,NaN,NaN,NaN,SAINT FRANCIS,NaN,...,NaN,NaN,NaN,NaN,1980.0,2004,NaN,NaN,45.41332,-93.26739
1,0.0,0.93,NaN,NaN,N,NaN,24457,NaN,SAINT FRANCIS,BETHEL,...,NaN,NaN,NaN,NaN,1974.0,2004,55005,NaN,45.41354,-93.2701
2,0.0,8.75,NaN,NaN,N,NaN,24442,NaN,SAINT FRANCIS,BETHEL,...,NaN,NaN,NaN,NaN,1969.0,2004,55005,NaN,45.41318,-93.27344
3,0.0,11.17,NaN,NaN,N,NaN,410,NaN,SAINT FRANCIS,BETHEL,...,NaN,NaN,NaN,NaN,1989.0,2004,55005,NaN,45.41167,-93.27684
4,0.0,14.46,NaN,NaN,N,NaN,480,NaN,SAINT FRANCIS,BETHEL,...,NaN,NaN,NaN,NaN,1995.0,2004,55070,NaN,45.41169,-93.27849


In [8]:
codes_with_complete_data = lake_stats.DNR_ID_Site_Number

In [10]:
rows_to_keep = (pd.Series(zip(lakes_2004.centroid_lat,lakes_2004.centroid_long))
          .map(ll_code_dict)
          .isin(codes_with_complete_data)
         )
rows_to_keep.sample(20)

1959044    False
834864     False
1456802    False
660621      True
604830     False
890712     False
681595     False
677627     False
1284129    False
1132335    False
63401       True
83718      False
1076867    False
580259     False
1331999    False
1441226    False
1270169    False
845057     False
494093     False
486247     False
dtype: bool

In [11]:
rows_to_keep.value_counts()

False    1786339
True      196036
dtype: int64

In [12]:
lakes_2004_condensed = (lakes_2004
                        >> filter_by(rows_to_keep)
                       )
lakes_2004_condensed.head()

,ACRES_DEED,ACRES_POLY,AGPRE_ENRD,AGPRE_EXPD,AG_PRESERV,BASEMENT,BLDG_NUM,BLOCK,CITY,CITY_USPS,...,XUSE1_DESC,XUSE2_DESC,XUSE3_DESC,XUSE4_DESC,YEAR_BUILT,Year,ZIP,ZIP4,centroid_lat,centroid_long
8,0.0,4.28,NaN,NaN,N,NaN,NaN,NaN,SAINT FRANCIS,NaN,...,NaN,NaN,NaN,NaN,0.0,2004,NaN,NaN,45.4117,-93.28091
21,0.0,1.64,NaN,NaN,N,NaN,NaN,NaN,SAINT FRANCIS,NaN,...,NaN,NaN,NaN,NaN,0.0,2004,NaN,NaN,45.41325,-93.35273
30,0.0,64.56,NaN,NaN,N,NaN,NaN,NaN,SAINT FRANCIS,NaN,...,NaN,NaN,NaN,NaN,0.0,2004,NaN,NaN,45.4101,-93.38513
38,0.0,39.25,NaN,NaN,N,NaN,NaN,NaN,SAINT FRANCIS,NaN,...,NaN,NaN,NaN,NaN,0.0,2004,NaN,NaN,45.41164,-93.39853
46,0.0,5.05,NaN,NaN,N,NaN,24436,1,SAINT FRANCIS,SAINT FRANCIS,...,NaN,NaN,NaN,NaN,1995.0,2004,55070,NaN,45.41222,-93.32077


In [13]:
lakes_2004_condensed.shape

(196036, 71)

In [9]:
@pipeable
def clean_parcel_file(df):
    new_df = (df
              >> select(cols_to_keep)
              >> mutate(lat_long = pd.Series(zip(df.centroid_lat,df.centroid_long)))
              >> mutate(lake_code = X.lat_long.map(ll_code_dict))
              >> filter_by(X.lake_code.isin(codes_with_complete_data))
              >> mutate(distance_to_lake = X.lat_long.map(ll_dist_dict),
                        lake_name = X.lake_code.map(code_name_dict))
              >> filter_by(X.distance_to_lake <= 1600)
             )
    return new_df

In [10]:
read_in_parcel_file = pipeable(lambda f: pd.read_csv(f,sep='|',dtype = common_parcel_types))

In [15]:
lakes_2004_complete = clean_parcel_file(lakes_2004)

In [16]:
lakes_2004_complete.head()

,ACRES_DEED,ACRES_POLY,AGPRE_ENRD,AG_PRESERV,BASEMENT,CITY,COOLING,DWELL_TYPE,EMV_BLDG,EMV_LAND,...,XUSE3_DESC,XUSE4_DESC,YEAR_BUILT,Year,centroid_lat,centroid_long,lat_long,lake_code,distance_to_lake,lake_name
8,0.0,4.28,NaN,N,NaN,SAINT FRANCIS,NaN,NaN,0.0,21800.0,...,NaN,NaN,0.0,2004,45.4117,-93.28091,"(45.4117, -93.28091)",82015900-01,187.280315,Forest Lake
21,0.0,1.64,NaN,N,NaN,SAINT FRANCIS,NaN,NaN,0.0,50400.0,...,NaN,NaN,0.0,2004,45.41325,-93.35273,"(45.41325, -93.35273)",19002900-01,742.058706,Lee Lake
30,0.0,64.56,NaN,N,NaN,SAINT FRANCIS,NaN,NaN,0.0,229646.0,...,NaN,NaN,0.0,2004,45.4101,-93.38513,"(45.4101, -93.38513)",82011602-01,1550.294020,Armstrong Lake
38,0.0,39.25,NaN,N,NaN,SAINT FRANCIS,NaN,NaN,0.0,140000.0,...,NaN,NaN,0.0,2004,45.41164,-93.39853,"(45.41164, -93.39853)",27062700-01,571.475034,Northwood Lake
56,0.0,331.82,NaN,N,NaN,SAINT FRANCIS,NaN,NaN,429784.0,1138600.0,...,NaN,NaN,1982.0,2004,45.40427,-93.49863,"(45.40427, -93.49863)",82009400-01,1474.663652,Colby Lake


In [17]:
lakes_2004_complete.shape

(122363, 43)

In [11]:
@dfpipe
def union_all(left_df, right_df, ignore_index=True):
    return pd.concat([left_df, right_df], ignore_index=ignore_index)

In [12]:
parcel_union = pd.DataFrame(columns=['ACRES_DEED', 'ACRES_POLY', 'AGPRE_ENRD', 'AG_PRESERV', 'BASEMENT',
       'CITY', 'COOLING', 'DWELL_TYPE', 'EMV_BLDG', 'EMV_LAND', 'FIN_SQ_FT',
       'GARAGE', 'GARAGESQFT', 'GREEN_ACRE', 'HOMESTEAD', 'LANDMARK',
       'OWN_ADD_L1', 'OWN_ADD_L2', 'OWN_ADD_L3', 'PARC_CODE', 'PIN',
       'SALE_VALUE', 'SPEC_ASSES', 'TAX_CAPAC', 'TAX_EXEMPT', 'TOTAL_TAX',
       'USE1_DESC', 'USE2_DESC', 'USE3_DESC', 'USE4_DESC', 'WSHD_DIST',
       'XUSE1_DESC', 'XUSE2_DESC', 'XUSE3_DESC', 'XUSE4_DESC', 'YEAR_BUILT',
       'Year', 'centroid_lat', 'centroid_long', 'lat_long', 'lake_code',
       'distance_to_lake', 'lake_name'])
parcel_union

,ACRES_DEED,ACRES_POLY,AGPRE_ENRD,AG_PRESERV,BASEMENT,CITY,COOLING,DWELL_TYPE,EMV_BLDG,EMV_LAND,...,XUSE3_DESC,XUSE4_DESC,YEAR_BUILT,Year,centroid_lat,centroid_long,lat_long,lake_code,distance_to_lake,lake_name


In [13]:
clean_parcels = [f >> read_in_parcel_file() >> clean_parcel_file() for f in files]

/home/ox6036qb/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (20,24) have mixed types. Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
/home/ox6036qb/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (20,24) have mixed types. Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
/home/ox6036qb/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (20,24) have mixed types. Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
/home/ox6036qb/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPy

In [14]:
for parcel in clean_parcels:
    parcel_union = parcel_union >> union_all(parcel)

for f in files:
    print("Beginning file {0}".format(f))
    df = pd.read_csv(f,sep='|',dtype = common_parcel_types)
    df_complete = clean_parcel_file(df)
    parcel_union = parcel_union >> union_all(df_complete)

In [15]:
parcel_union.sample(20)

,ACRES_DEED,ACRES_POLY,AGPRE_ENRD,AG_PRESERV,BASEMENT,CITY,COOLING,DWELL_TYPE,EMV_BLDG,EMV_LAND,...,XUSE3_DESC,XUSE4_DESC,YEAR_BUILT,Year,centroid_lat,centroid_long,lat_long,lake_code,distance_to_lake,lake_name
1000648,0.0,9.37,NaN,N,Y,CITY OF HUGO,Y,Single-Family / Owner Occupied,164300.0,132800.0,...,NaN,NaN,1983.0,2011,45.20551,-92.94532,"(45.20551, -92.94532)",82009700-01,1507.706506,La Lake
661559,0.0,0.24,1899-12-30,N,NaN,BROOKLYN CENTER,Forced Air,NaN,155300.0,64600.0,...,NaN,NaN,1968.0,2009,45.08649,-93.28488,"(45.08649, -93.28488)",27007000-01,1142.613838,Mitchell Lake
847057,0.0,0.32,NaN,N,NaN,ST. LOUIS PARK,Forced Air,NaN,260200.0,83800.0,...,NaN,NaN,1985.0,2010,44.96471,-93.38093,"(44.96471, -93.38093)",19034800-01,1088.854685,Valley Lake
825913,0.0,0.69,NaN,N,NaN,EAGAN,NaN,NaN,289700.0,150000.0,...,NaN,NaN,0.0,2010,44.84916,-93.13952,"(44.84916, -93.13952)",82015900-01,412.456549,Forest Lake
640592,0.0,0.25,NaN,N,NaN,APPLE VALLEY,NaN,S.FAM.RES,119300.0,57600.0,...,NaN,NaN,1966.0,2009,44.72488,-93.23512,"(44.72488, -93.23512)",19002900-01,656.683808,Lee Lake
305470,0.33,0.0,NaN,NaN,NaN,STILLWATER CITY,NaN,NaN,467200.0,115800.0,...,NaN,NaN,2000.0,2006,45.04996,-92.86039,"(45.04996, -92.86039)",27062700-01,1097.153899,Northwood Lake
904296,0.15,0.15,NaN,N,Y,SAINT PAUL,CENTRAL,TWO FAMILY DWELLING,198100.0,120100.0,...,NaN,NaN,1912.0,2011,44.94431,-93.18488,"(44.94431, -93.18488)",70002600-01,103.130063,Lower Prior Lake
1325501,0.0,0.72,NaN,N,NaN,TOWN OF WEST LAKELAND,NaN,NaN,988100.0,325300.0,...,NaN,NaN,0.0,2014,44.95237,-92.80648,"(44.95237, -92.80648)",82010100-01,39.139376,DeMontreville Lake
598223,0.0,0.15,1899-12-30,NaN,NaN,MINNEAPOLIS,NaN,NaN,151700.0,45800.0,...,NaN,NaN,1954.0,2008,45.04061,-93.31898,"(45.04061, -93.31898)",27004201-01,403.880988,Twin Lake
456129,0.0,0.38,NaN,NaN,NaN,BROOKLYN PARK,NaN,NaN,238900.0,83000.0,...,NaN,NaN,1999.0,2007,45.13085,-93.32673,"(45.13085, -93.32673)",82011602-01,902.041369,Armstrong Lake


In [16]:
parcel_union.shape

(1394880, 43)

In [17]:
parcel_union.to_csv("./data/union_of_parcel_data.csv")